# Testing RPCA-PGSG on tree dataset
Reads in ./data/trees.bin

Writes output to ./results/

In [1]:
include("./PGSG.jl")
using PGSG

In [ ]:
##### load data #####
println("load data...")
n,m = 20480, 102;
fid = open("./data/trees.bin","r")
A   = read(fid, UInt8, n*m)
A   = reshape(A, n, m)
close(fid)

A   = A/255

##### set up parameters #####
k   = 10; #number of componets in low rank model

# svd initialization
println("svd initialization...")
UΣV,nconv,niter,nmult,resid = svds(A,nsv=k,tol=1e-12)
U   = diagm(sqrt(UΣV[:S]))*UΣV[:U].';
V   = diagm(sqrt(UΣV[:S]))*UΣV[:V];
for I in eachindex(U)
    U[I] < 0.0 ? U[I] = 0.0 :
    U[I] > 1.0 ? U[I] = 1.0 : continue
end
for I in eachindex(V)
    V[I] < 0.0 ? V[I] = 0.0 :
    V[I] > 1.0 ? V[I] = 1.0 : continue
end

##### apply solver #####
println("start solver...")
RPCA_PF_PGSG(U,V,A,0.1,10);


load data...
svd initialization...
start solver...
Inner loop finished
Inner loop finished


# Write output to Results directory

In [ ]:
##### recover S and save data #####
m,n = size(A);
println("recover S...")
L = U.'*V;
r = A - L;
κσ = 1;
# κσ = κ;
S = zeros(m,n);
for I in eachindex(S)
    r[I] >  κσ ? S[I] = r[I] - κσ :
    r[I] < -κσ ? S[I] = r[I] + κσ : continue;
end

##### save results #####
println("save data...")
fid = open("./results/trees_Huber_UV.bin","w");
write(fid,U,V);
close(fid);
fid = open("./results/trees_Huber_LS.bin","w");
write(fid,L,S);
close(fid);
fid = open("./results/trees_Huber_R.bin","w");
write(fid,r);
close(fid);


# Generate Pictures out of Results directory

In [ ]:
#======= Analysis the Results from RPCA =======#
using PyPlot

m,n = 20480, 102;
mp,np = 128, 160;

# load data
fid = open("./results/trees_Huber_LS.bin","r");
L   = read(fid, Float64, m*n);
S   = read(fid, Float64, m*n);
close(fid)
fid = open("./results/trees_Huber_R.bin","r");
R   = read(fid, Float64, m*n);
close(fid)
L   = reshape(L,m,n);
S   = reshape(S,m,n);
R   = reshape(R,m,n);
P   = copy(S);

id  = [101,102]
L   = L[:,id]
S   = S[:,id]
R   = R[:,id]
P   = P[:,id]

for I in eachindex(L)
    L[I] < 0.0 ? L[I] = 0.0 :
    L[I] > 1.0 ? L[I] = 1.0 : continue
end
for I in eachindex(S)
    S[I] = abs(S[I])
    R[I] = abs(R[I])
    # S[I] == 0.0 || (S[I] = 1.0)
    # R[I] == 0.0 || (R[I] = 1.0)
end

for I in eachindex(P)
    P[I] == 0.0 || (P[I] = 1.0)
end

# plot result

for i in 1:length(id)
    imshow([reshape(L[:,i],mp,np);reshape(R[:,i],mp,np);reshape(P[:,i],mp,np)], cmap="gist_gray");
    axis("off")
    savefig(@sprintf("./results/trees_h%06d.eps",i+2),bbox_inches="tight",transparent=true);
end